##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Como migrar LoggingTensorHook e StopAtStepHook para callbacks Keras

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/guide/migrate/logging_stop_hook"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/guide/migrate/logging_stop_hook.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/guide/migrate/logging_stop_hook.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fonte no GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/guide/migrate/logging_stop_hook.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
</table>

No TensorFlow 1, você usa `tf.estimator.LoggingTensorHook` para monitorar e registrar tensores, enquanto que `tf.estimator.StopAtStepHook` ajuda a interromper o treinamento numa passo específico ao treinar com `tf.estimator.Estimator`. Este notebook demonstra como migrar dessas APIs para seus equivalentes no TensorFlow 2 usando callbacks Keras personalizados (`tf.keras.callbacks.Callback`) com `Model.fit`.

Os [callbacks](https://www.tensorflow.org/guide/keras/custom_callback) do Keras são objetos chamados em diferentes pontos durante o treinamento/avaliação/previsão nas APIs integradas do Keras `Model.fit`/`Model.evaluate`/`Model.predict`. Você pode aprender mais sobre callbacks na documentação da API `tf.keras.callbacks.Callback`, bem como nos guias [Escrevendo seus próprios callbacks](../..guide/keras/custom_callback.ipynb/) e [Treinamento e avaliação com os métodos integrados](https://www.tensorflow.org/guide/keras/train_and_evaluate) (na seção *Usando callbacks*). Para migrar de `SessionRunHook` no TensorFlow 1 para callbacks do Keras no TensorFlow 2, confira o guia [Como migrar treinamento com lógica assistida](sessionrunhook_callback.ipynb).

## Configuração

Comece com os imports e um dataset simples para fins de demonstração:

In [ ]:
import tensorflow as tf
import tensorflow.compat.v1 as tf1

In [ ]:
features = [[1., 1.5], [2., 2.5], [3., 3.5]]
labels = [[0.3], [0.5], [0.7]]

# Define an input function.
def _input_fn():
  return tf1.data.Dataset.from_tensor_slices((features, labels)).batch(1)

## TensorFlow 1: registre os tensores e interrompa o treinamento com as APIs tf.estimator

No TensorFlow 1, você define vários hooks para controlar o comportamento do treinamento. Depois você passa esses hooks para o `tf.estimator.EstimatorSpec`.

No exemplo abaixo:

- Para monitorar/registrar tensores, por exemplo, pesos ou perdas de modelos, você usa `tf.estimator.LoggingTensorHook` (`tf.train.LoggingTensorHook` é um alias para a mesma classe).
- Para interromper o treinamento num determinado passo, use `tf.estimator.StopAtStepHook` (`tf.train.StopAtStepHook` é um alias para a mesma classe).

In [ ]:
def _model_fn(features, labels, mode):
  dense = tf1.layers.Dense(1)
  logits = dense(features)
  loss = tf1.losses.mean_squared_error(labels=labels, predictions=logits)
  optimizer = tf1.train.AdagradOptimizer(0.05)
  train_op = optimizer.minimize(loss, global_step=tf1.train.get_global_step())

  # Define the stop hook.
  stop_hook = tf1.train.StopAtStepHook(num_steps=2)

  # Access tensors to be logged by names.
  kernel_name = tf.identity(dense.weights[0])
  bias_name = tf.identity(dense.weights[1])
  logging_weight_hook = tf1.train.LoggingTensorHook(
      tensors=[kernel_name, bias_name],
      every_n_iter=1)
  # Log the training loss by the tensor object.
  logging_loss_hook = tf1.train.LoggingTensorHook(
      {'loss from LoggingTensorHook': loss},
      every_n_secs=3)

  # Pass all hooks to `EstimatorSpec`.
  return tf1.estimator.EstimatorSpec(mode,
                                     loss=loss,
                                     train_op=train_op,
                                     training_hooks=[stop_hook,
                                                     logging_weight_hook,
                                                     logging_loss_hook])

estimator = tf1.estimator.Estimator(model_fn=_model_fn)

# Begin training.
# The training will stop after 2 steps, and the weights/loss will also be logged.
estimator.train(_input_fn)

## TensorFlow 2: registre os tensores e interrompa o treinamento com callbacks personalizados e Model.fit

No TensorFlow 2, quando você usa o Keras `Model.fit` integrado (ou `Model.evaluate`) para treinamento/avaliação, pode configurar o monitoramento do tensor e a interrupção do treinamento definindo callbacks do Keras (`tf.keras.callbacks.Callback`) personalizados. Em seguida, você os passa para o parâmetro `callbacks` de `Model.fit` (ou `Model.evaluate`). (Saiba mais no guia [Como escrever seus próprios callbacks](../..guide/keras/custom_callback.ipynb))

No exemplo abaixo:

- Para recriar as funcionalidades de `StopAtStepHook`, defina um retorno de chamada personalizado (denominado `StopAtStepCallback` abaixo) onde você substitui o método `on_batch_end` para interromper o treinamento após um determinado número de passos.
- Para recriar o comportamento de `LoggingTensorHook`, defina um callback personalizado (`LoggingTensorCallback`) onde você registra e gera os tensores registrados manualmente, pois não há suporte para o acesso a tensores por nomes. Você também pode implementar a frequência de registro do log dentro do callback personalizado. O exemplo abaixo imprimirá os pesos a cada dois passos. Outras estratégias, como registrar o log a cada N segundos, também são possíveis.

In [ ]:
class StopAtStepCallback(tf.keras.callbacks.Callback):
  def __init__(self, stop_step=None):
    super().__init__()
    self._stop_step = stop_step

  def on_batch_end(self, batch, logs=None):
    if self.model.optimizer.iterations >= self._stop_step:
      self.model.stop_training = True
      print('\nstop training now')

class LoggingTensorCallback(tf.keras.callbacks.Callback):
  def __init__(self, every_n_iter):
      super().__init__()
      self._every_n_iter = every_n_iter
      self._log_count = every_n_iter

  def on_batch_end(self, batch, logs=None):
    if self._log_count > 0:
      self._log_count -= 1
      print("Logging Tensor Callback: dense/kernel:",
            model.layers[0].weights[0])
      print("Logging Tensor Callback: dense/bias:",
            model.layers[0].weights[1])
      print("Logging Tensor Callback loss:", logs["loss"])
    else:
      self._log_count -= self._every_n_iter

Quando terminar, passe os novos callbacks — `StopAtStepCallback` e `LoggingTensorCallback` — para o parâmetro `callbacks` de `Model.fit`:

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((features, labels)).batch(1)
model = tf.keras.models.Sequential([tf.keras.layers.Dense(1)])
optimizer = tf.keras.optimizers.Adagrad(learning_rate=0.05)
model.compile(optimizer, "mse")

# Begin training.
# The training will stop after 2 steps, and the weights/loss will also be logged.
model.fit(dataset, callbacks=[StopAtStepCallback(stop_step=2),
                              LoggingTensorCallback(every_n_iter=2)])

## Próximos passos

Saiba mais sobre callbacks em:

- Documentação da API: `tf.keras.callbacks.Callback`
- Guia: [Escrevendo seus próprios callbacks](../..guide/keras/custom_callback.ipynb/)
- Guia: [Treinamento e avaliação com os métodos integrados](https://www.tensorflow.org/guide/keras/train_and_evaluate) (a seção *Usando callbacks* )

Você talvez também ache úteis os seguintes recursos relacionados à migração:

- O [Guia de migração de paradas antecipadas](early_stopping.ipynb) : `tf.keras.callbacks.EarlyStopping` é um callback integrado para paradas antecipadas (early stopping).
- O [Guia de migração do TensorBoard](tensorboard.ipynb): o TensorBoard permite rastrear e exibir métricas
- O guia [Treinamento com migração de lógica assistida](sessionrunhook_callback.ipynb): de `SessionRunHook` no TensorFlow 1 para callbacks do Keras no TensorFlow 2